In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df = catalog.load('raw/openaire/researchproduct_dev#parquet')

[09/08/25 10:48:24] INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=6408;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=990483;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [3]:
df

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,publicationDate,publisher,embargoEndDate,sources,formats,contributors,coverages,bestAccessRight,container,documentationUrls,codeRepositoryUrl,programmingLanguage,contactPeople,contactGroups,tools,size,version,geoLocations,id,originalIds,pids,dateOfCollection,lastUpdateTimeStamp,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
0,"[{'fullName': 'Susevich, María Laura', 'name':...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Dicistrovirus from the pollinator community fo...,None,[Los Dicistrovirus son una familia de virus qu...,2022-08-31,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::319dc88111c9b2d6021228590e79130a,[50|4dc99724cf04::319dc88111c9b2d6021228590e79...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLPAM', 'id': 'openorgs____::db...","[{'code': 'knowmad', 'label': 'Knowmad Institu...",[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
1,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Variability in the growth rates of Saanen kids...,None,[This study was carried out to determine the v...,2023-12-18,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"[oai:ojs.csi.unmsm:article/25240, 50|4dc99724c...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,[{'acronym': 'Universidad Nacional de La Pampa...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
2,"[{'fullName': 'Paredes A., Manuel', 'name': 'M...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Effects of dietary inclusion of alfalfa meal o...,None,[Se utilizaron 240 pollos de engorde hembras H...,2020-06-20,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::fd4ac9e4a4b67e70441e5d033c144e1e,[50|4dc99724cf04::fd4ac9e4a4b67e70441e5d033c14...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,[{'acronym': 'Universidad Nacional de La Pampa...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
3,"[{'fullName': 'Evangelista, Carolina', 'name':...",None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Estudio de la viabilidad del algoritmo super-t...,None,[En este trabajo se presenta el diseño de un c...,2013-01-01,None,None,[reponame:Digital.CSIC. Repositorio Institucio...,None,"[European Commission, Ministerio de Economía y...",None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",None,None,None,None,None,None,Non

In [4]:
df_research_author = df[['id','authors']].explode('authors').reset_index(drop=True)

In [5]:
df_research_author

,id,authors
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"{'fullName': 'Susevich, María Laura', 'name': ..."
1,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"{'fullName': 'Echeverria, Maria Gabriela', 'na..."
2,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"{'fullName': 'Origlia, Javier', 'name': 'Javie..."
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"{'fullName': 'Steffen, Kevin Denis', 'name': '..."
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"{'fullName': 'Gortari Castillo, Lihuel', 'name..."
...,...,...
2682,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"{'fullName': 'Pera, María Sol', 'name': 'Marí..."
2683,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"{'fullName': 'Perren, Gabriel Ignacio', 'name'..."
2684,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"{'fullName': 'Vázquez, Rubén Ángel', 'name': '..."
2685,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"{'fullName': 'Cruzado, Alicia', 'name': 'Alici..."


In [6]:
df_authors = pd.json_normalize(df_research_author['authors'])

In [7]:
df_authors

,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance
0,"Susevich, María Laura",María Laura,NaN,1,Susevich,NaN,NaN,NaN
1,"Echeverria, Maria Gabriela",Maria Gabriela,NaN,2,Echeverria,NaN,NaN,NaN
2,"Origlia, Javier",Javier,NaN,3,Origlia,NaN,NaN,NaN
3,"Steffen, Kevin Denis",Kevin Denis,NaN,1,Steffen,NaN,NaN,NaN
4,"Gortari Castillo, Lihuel",Lihuel,NaN,2,Gortari Castillo,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2682,"Pera, María Sol",María Sol,NaN,2,Pera,NaN,NaN,NaN
2683,"Perren, Gabriel Ignacio",Gabriel Ignacio,NaN,3,Perren,NaN,NaN,NaN
2684,"Vázquez, Rubén Ángel",Rubén Ángel,NaN,4,Vázquez,NaN,NaN,NaN
2685,"Cruzado, Alicia",Alicia,NaN,5,Cruzado,NaN,NaN,NaN


In [8]:
df_research_author = pd.concat([df_research_author['id'], df_authors], axis=1)


In [9]:
df_research_author

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Susevich, María Laura",María Laura,NaN,1,Susevich,NaN,NaN,NaN
1,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Echeverria, Maria Gabriela",Maria Gabriela,NaN,2,Echeverria,NaN,NaN,NaN
2,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Origlia, Javier",Javier,NaN,3,Origlia,NaN,NaN,NaN
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Steffen, Kevin Denis",Kevin Denis,NaN,1,Steffen,NaN,NaN,NaN
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Gortari Castillo, Lihuel",Lihuel,NaN,2,Gortari Castillo,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2682,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Pera, María Sol",María Sol,NaN,2,Pera,NaN,NaN,NaN
2683,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Perren, Gabriel Ignacio",Gabriel Ignacio,NaN,3,Perren,NaN,NaN,NaN
2684,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Vázquez, Rubén Ángel",Rubén Ángel,NaN,4,Vázquez,NaN,NaN,NaN
2685,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Cruzado, Alicia",Alicia,NaN,5,Cruzado,NaN,NaN,NaN


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [10]:
def _pick_load_dt(df: pd.DataFrame):
    # Si hay una sola fecha en el batch, usala; si hay varias, quedate con la más reciente;
    # si no hay, hoy.
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()

In [11]:
def openaire_land_researchproduct_authors(df: pd.DataFrame)-> pd.DataFrame:

    load_dt = _pick_load_dt(df)

    df_research_author = df[['id','authors']].explode('authors').reset_index(drop=True)

    df_authors = pd.json_normalize(df_research_author['authors'])

    df_research_author = pd.concat([df_research_author['id'], df_authors], axis=1)

    df_research_author['load_datetime'] = load_dt

    return df_research_author


In [12]:
df_researchproduct_author = openaire_land_researchproduct_authors(df)

In [13]:
df_researchproduct_author

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance,load_datetime
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Susevich, María Laura",María Laura,NaN,1,Susevich,NaN,NaN,NaN,2025-09-05
1,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Echeverria, Maria Gabriela",Maria Gabriela,NaN,2,Echeverria,NaN,NaN,NaN,2025-09-05
2,4dc99724cf04::319dc88111c9b2d6021228590e79130a,"Origlia, Javier",Javier,NaN,3,Origlia,NaN,NaN,NaN,2025-09-05
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Steffen, Kevin Denis",Kevin Denis,NaN,1,Steffen,NaN,NaN,NaN,2025-09-05
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Gortari Castillo, Lihuel",Lihuel,NaN,2,Gortari Castillo,NaN,NaN,NaN,2025-09-05
...,...,...,...,...,...,...,...,...,...,...
2682,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Pera, María Sol",María Sol,NaN,2,Pera,NaN,NaN,NaN,2025-09-05
2683,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Perren, Gabriel Ignacio",Gabriel Ignacio,NaN,3,Perren,NaN,NaN,NaN,2025-09-05
2684,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Vázquez, Rubén Ángel",Rubén Ángel,NaN,4,Vázquez,NaN,NaN,NaN,2025-09-05
2685,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"Cruzado, Alicia",Alicia,NaN,5,Cruzado,NaN,NaN,NaN,2025-09-05
